In [30]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.3.0


In [31]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [32]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [33]:
train_data

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [54]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


## Note : Word index starts from 1 which is oov_token. When padded is done, if text is not long enough, 0 will be added to the beginning/end array. So in decode_review function, it will return '*' when it see 0, since '0' has no value in reverse_word_index dictionary.

---



In [55]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    print("text is ",text)
    return ' '.join([reverse_word_index.get(i, '*') for i in text])

print( "OOV Means word not in tokenizer. * represents pad that is added later.")
print()
print(decode_review(padded[3]))
print(training_sentences[3])

OOV Means word not in tokenizer. * represents pad that is added later.

text is  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   12    7    2  241    5   20   16    4 8776 2707 2653
   52    2  358    5    2  180   68  138 1401   17   92  203  968   15
   23    1   81    4  192 3109 3037    3    1   16    4  376    5  632
  387  352   37 6355    3 5559 1928   15  208 8596 3433    2  112  365
   48   24   55    1    6 1657   55 1818 4312   41    4 2263    3 1908
    1  141  160  780  111   31   92  116    4  221   20    9  173  279
    3   29   13 1012    2 2814   10 1985]
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gentl

In [56]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


## Embedding layer is basically lookup table. Param indicates there are 1000(vocabsize) row with 16 ( embedding size) column in look up table.
As every sentence limited with max 120 word. it can 120*16 as a output shape.

In [57]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4889 - accuracy: 0.7444 - val_loss: 0.3448 - val_accuracy: 0.8493
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2323 - accuracy: 0.9127 - val_loss: 0.3744 - val_accuracy: 0.8375
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0846 - accuracy: 0.9781 - val_loss: 0.4627 - val_accuracy: 0.8243
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0218 - accuracy: 0.9972 - val_loss: 0.5259 - val_accuracy: 0.8238
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0056 - accuracy: 0.9996 - val_loss: 0.5872 - val_accuracy: 0.8253
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6488 - val_accuracy: 0.8228
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.8129e-04 - accuracy: 1.0000 - val_loss: 0.6814 - val_accuracy: 0.8258
Ep

In [58]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [59]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [40]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Prediction Part

In [86]:
print(training_sentences[5])

sentence1 = training_sentences[5]
sequence1 = tokenizer.texts_to_sequences([sentence])
print(sequence1)

pad1 = pad_sequences(sequence1,maxlen=max_length, truncating=trunc_type)


a = model.predict(pad1)
print(a)
print(a>0.5)

This is a film which should be seen by anybody interested in, effected by, or suffering from an eating disorder. It is an amazingly accurate and sensitive portrayal of bulimia in a teenage girl, its causes and its symptoms. The girl is played by one of the most brilliant young actresses working in cinema today, Alison Lohman, who was later so spectacular in 'Where the Truth Lies'. I would recommend that this film be shown in all schools, as you will never see a better on this subject. Alison Lohman is absolutely outstanding, and one marvels at her ability to convey the anguish of a girl suffering from this compulsive disorder. If barometers tell us the air pressure, Alison Lohman tells us the emotional pressure with the same degree of accuracy. Her emotional range is so precise, each scene could be measured microscopically for its gradations of trauma, on a scale of rising hysteria and desperation which reaches unbearable intensity. Mare Winningham is the perfect choice to play her mot

In [85]:
#My Test Set by just an input
mytestdata = np.array(["It was great, I liked it.","I hate it , disgusting!","Shit ","best great good movie","great best movie, amazing"])
anothersequences = tokenizer.texts_to_sequences(mytestdata)
padded2 = pad_sequences(anothersequences,maxlen=120, truncating=trunc_type)
another_prediction = model.predict(padded2)

#My Test Set results
another_prediction_result = another_prediction > 0.5
print(another_prediction)
print(another_prediction_result)



[[0.8459643 ]
 [0.00949743]
 [0.09955162]
 [0.89622116]
 [0.9958124 ]]
[[ True]
 [False]
 [False]
 [ True]
 [ True]]
